# GitHub FAQ Generator

This notebook demonstrates how to generate a FAQ from GitHub issues using an LLM.

In [3]:
import os
from github import Github
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# GitHub API setup
github_token = os.getenv("GITHUB_TOKEN")
g = Github(github_token)

print("GitHub API setup complete.")

GitHub API setup complete.


In [26]:
def ask_llm(input):
    print(input)
    # Placeholder function for LLM integration
    # In a real implementation, this would call your LLM service
    # The LLM should process all issues at once and return a list of FAQ entries
    return [
        {"question": "What is this project about?", "answer": "This is a placeholder answer about the project."},
        {"question": "How do I integrate this in my app?", "answer": "This is a placeholder answer about integration."},
        {"question": "What platforms are supported?", "answer": "This is a placeholder answer about supported platforms."}
    ]

print("LLM placeholder function defined.")

LLM placeholder function defined.


In [5]:
def fetch_issues(repo_name):
    repo = g.get_repo(repo_name)
    issues = repo.get_issues(state="all")
    return issues

print("Issue fetching function defined.")

Issue fetching function defined.


In [6]:
issues = fetch_issues("britannio/in_app_review")

In [13]:
issue_tuples_with_comments = [(issue.title, issue.body, [comment.body for comment in issue.get_comments()]) for issue in issues]
# issue_tuples = [(issue.title, issue.body) for issue in issues]


In [24]:
ref_issue = issue_tuples_with_comments[1]
print("TITLE: " + ref_issue[0])
print("BODY: ")
print(ref_issue[1].strip())

print('---')
for i in ref_issue[2]:
    print(i)
    print('---')

TITLE: User already rated
BODY: 
Good morning,

There is only one information that I could not find anywhere, if the user already submitted the rating, will the pop-up appear to them again? Of course in case they did not exhaust the quota.

Thank you for the amazing package!
---
Any feedback?
---
@JadyMcIntyre  I searched in the documentations for iOS and Android and I found the answer.

**for Apple**
https://developer.apple.com/design/human-interface-guidelines/ratings-and-reviews

> When you identify places in your experience where it makes sense to ask for feedback, the system checks for previous feedback and — if there isn’t any — displays an in-app prompt that asks for a rating and an optional written review.

for Android
https://developer.android.com/guide/playcore/in-app-review/test

> The user account does not currently have a review for the app.


I hope this helps.
---
@DorraY This definitely does help thank you 🙏 

do you by any chance know if the _inAppReview.isAvailable();

In [33]:
def generate_faq(repo_name):
    # issues = fetch_issues(repo_name)
    issues = issue_tuples_with_comments
    
    # Prepare all issue content for the LLM
    all_issues_content = []
    for issue in issues:
        issue_content = f"Title: {issue[0]}\nBody: {issue[1]}\nComments:\n"
        for comment in issue[2]:
            issue_content += f"{comment}\n---\n" 
        all_issues_content.append(issue_content)
    
    # Join all issue content into a single string
    combined_issues = "\n\n".join(all_issues_content)
    
    # Process all issues at once using the LLM
    faq_entries = ask_llm(combined_issues)
    
    # Generate a simple markdown FAQ
    faq_content = "# FAQ\n\n"
    for entry in faq_entries:
        faq_content += f"## {entry['question']}\n\n{entry['answer']}\n\n"
    
    return faq_content

print("FAQ generation function defined.")

FAQ generation function defined.


In [34]:
# Set the repository name
repo_name = "britannio/in_app_review"

# Generate the FAQ
faq = generate_faq(repo_name)

# Display the generated FAQ
print(faq)

Title: Duplicate class: version 2.0.9 is conflicted with  com.google.android.play:core:1.10.3
Body: ```
A failure occurred while executing com.android.build.gradle.internal.tasks.CheckDuplicatesRunnable
   > Duplicate class com.google.android.play.core.common.IntentSenderForResultStarter found in modules jetified-core-1.10.3-runtime (com.google.android.play:core:1.10.3) and jetified-core-common-2.0.2-runtime (com.google.android.play:core-common:2.0.2)
     Duplicate class com.google.android.play.core.common.LocalTestingException found in modules jetified-core-1.10.3-runtime (com.google.android.play:core:1.10.3) and jetified-core-common-2.0.2-runtime (com.google.android.play:core-common:2.0.2)
     Duplicate class com.google.android.play.core.common.PlayCoreDialogWrapperActivity found in modules jetified-core-1.10.3-runtime (com.google.android.play:core:1.10.3) and jetified-core-common-2.0.2-runtime (com.google.android.play:core-common:2.0.2)
     Duplicate class com.google.android.play

In [ ]:
# Save the FAQ to a file
with open("faq.md", "w") as f:
    f.write(faq)

print("FAQ generated and saved to faq.md")

## Next Steps

1. Implement the actual LLM integration in the `ask_llm` function.
2. Experiment with different ways of formatting the issue content for the LLM.
3. Add error handling and logging.
4. Implement rate limiting to avoid hitting GitHub API limits.
5. Add functionality to filter issues based on labels or other criteria.
6. Extend the notebook to include the API implementation using Flask or FastAPI.

Okay, I have a system for scraping all issues and their comments.
With Claude & function calling I should be able to do this:
(issue, [comment]) -> [(question, answer)]
Then I should be able to do a global 'reduce' where I collect all of these pairs and run it through a LLM again to create a final list of pairs.


W